In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn import linear_model
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_score
pd.options.mode.chained_assignment = None
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from impyute.imputation.cs import fast_knn


# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk(r'E:\Documents\Kaggle Projects\Categorical feature encoding'):
    for filename in filenames:
        file=os.path.join(dirname, filename)
        if filename=='train.csv':
            train= file
        if filename=='test.csv':
            test= file

# Any results you write to the current directory are saved as output.


# Reduce memory usage

In [2]:
'''Reduce memory usage'''
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df
#####################################
#df_train=reduce_mem_usage(df_train)
#df_test=reduce_mem_usage(df_test)
#####################################
'''Variable Description'''
def description(df):
    #print(f"Dataset Shape: {df.shape}")
    summary = pd.DataFrame(df.dtypes,columns=['dtypes'])
    summary = summary.reset_index()
    summary['Name'] = summary['index']
    summary = summary[['Name','dtypes']]
    summary['Missing'] = df.isnull().sum().values    
    summary['Uniques'] = df.nunique().values
    summary['First Value'] = df.iloc[0].values
    summary['Second Value'] = df.iloc[1].values
    summary['Third Value'] = df.iloc[2].values
    return summary

In [3]:
def label_encoding(label_df,label_col):
    if label_col == None:
        label_df = label_df.apply(lambda series: pd.Series(
        LabelEncoder().fit_transform(series[series.notnull()]),
        index=series[series.notnull()].index))
    else:
        label_df[label_col] = label_df[label_col].apply(lambda series: pd.Series(
        LabelEncoder().fit_transform(series[series.notnull()]),
        index=series[series.notnull()].index))
    return label_df
            
def one_hot_encoding(one_df,one_col):
    one_df = pd.concat([one_df,pd.get_dummies(one_df[one_col], prefix=one_col)],axis=1).drop(one_col,axis=1)
    return one_df

In [4]:
def logreg(x_train,y_train,x_val):
    logreg = LogisticRegression(solver='lbfgs')
    logreg.fit(x_train, y_train)
    Y_pred = logreg.predict(x_test)
    logreg.score(x_train, y_train)
    acc_log = round(logreg.score(x_train, y_train) * 100, 2)
    return Y_pred ,acc_log

def sgd(x_train,y_train,x_val):
    sgd = linear_model.SGDClassifier(max_iter=5, tol=None)
    sgd.fit(x_train, y_train)
    Y_pred = sgd.predict(x_val)
    sgd.score(x_train, y_train)
    acc_sgd = round(sgd.score(x_train, y_train) * 100, 2)
    return Y_pred ,acc_sgd

def rnd_forest(x_train,y_train,x_val):
    random_forest = RandomForestClassifier(n_estimators=100)
    random_forest.fit(x_train, y_train)
    Y_pred = random_forest.predict(x_val)
    random_forest.score(x_train, y_train)
    acc_random_forest = round(random_forest.score(x_train,y_train) * 100, 2)
    return Y_pred ,acc_random_forest

def knn(x_train,y_train,x_val):
    knn = KNeighborsClassifier(n_neighbors = 3) 
    knn.fit(x_train, y_train)  
    Y_pred = knn.predict(x_val) 
    acc_knn = round(knn.score(x_train, y_train) * 100, 2)
    return Y_pred ,acc_knn

def gaussian(x_train,y_train,x_val):
    gaussian = GaussianNB() 
    gaussian.fit(x_train, y_train)  
    Y_pred = gaussian.predict(x_val)  
    acc_gaussian = round(gaussian.score(x_train,y_train) * 100, 2)
    return Y_pred ,acc_gaussian

def perceptron (x_train,y_train,x_val):
    perceptron.fit(x_train, y_train)
    Y_pred = perceptron.predict(x_val)
    acc_perceptron = round(perceptron.score(x_train, y_train) * 100, 2)
    return Y_pred ,acc_perceptron

def linearsvc(x_train,y_train,x_val):
    linear_svc = LinearSVC()
    linear_svc.fit(x_train, y_train)
    Y_pred = linear_svc.predict(x_val)
    acc_linear_svc = round(linear_svc.score(x_train, y_train) * 100, 2)
    return Y_pred ,acc_linear_svc

def decisiontree(x_train,y_train,x_val):
    decision_tree = DecisionTreeClassifier() 
    decision_tree.fit(x_train, y_train)  
    Y_pred = decision_tree.predict(x_val) 
    acc_decision_tree = round(decision_tree.score(x_train, y_train) * 100, 2)
    return Y_pred ,acc_decision_tree


def featureimportance(x_train, y_train):
    random_forest = RandomForestClassifier(n_estimators=100)
    random_forest.fit(x_train, y_train)
    importances = pd.DataFrame({'feature':x_train.columns,'importance':np.round(random_forest.feature_importances_,3)})
    importances = importances.sort_values('importance',ascending=False).set_index('feature')
    return importances